In [2]:
import json
import logging
import warnings
import requests
import pandas as pd
from edgar import *
from tqdm import tqdm
from urllib3 import Retry
from edgar.xbrl import XBRLS
from datetime import datetime
from requests.adapters import HTTPAdapter
from bs4 import BeautifulSoup


warnings.simplefilter("ignore")
logging.getLogger().setLevel(logging.WARNING)
logging.getLogger("openai").setLevel(logging.ERROR)
logging.getLogger("urllib3").setLevel(logging.ERROR)
request = requests.Session()
retries = Retry(
    total=3,  # 最多重試 5 次
    backoff_factor=1,  # 每次重試的延遲時間指數增長（1s, 2s, 4s, 8s...）
    status_forcelist=[403, 429, 500, 502, 503, 504],  # 針對這些 HTTP 狀態碼進行重試
)
request.mount("http://", HTTPAdapter(max_retries=retries))
request.mount("https://", HTTPAdapter(max_retries=retries))
headers = {"User-Agent": "ansa ansa1019@gmail.com"}


# 公司名單
companies_range = 100
df_companies = pd.read_csv("sp500_companies.csv")
companies = df_companies.drop_duplicates(subset=["Shortname"], keep="first")[
    "Symbol"
].values[:companies_range]
with open("companies.json", "r", encoding="utf-8") as f:
    data = json.load(f)
missing_companies = [entry["ticker"] for entry in data["missing_companies"]]
find_cik = {entry["ticker"]: entry["cik"] for entry in data["find_cik"]}
companies = [c for c in companies if c not in missing_companies]

# 參數設定
set_identity("ansa ansa1019@gmail.com")
search_queries = ["IT capability", "organizational resilience"]
keywords = {q: [q] for q in search_queries}
report_item = {"paper_7": ["7"], "paper_17": ["1A", "7"]}
report_papers = "papers.json"
financials_file = "financials.csv"
report_year = [2014, 2023]
minlen = 1500
keyword_num = 30

In [15]:
# def
def extract_financials(cik, year, extract_type=None):
    company = Company(cik)
    df = company.get_facts().to_pandas()
    df = df[(df["form"] == "10-K") & (df["namespace"] == "us-gaap")]
    filings = company.get_filings(form=["10-K"], date=f"{year}-01-01:{year+1}-12-31")
    if not filings:
        return None

    # 選取符合該年度報告的 filing
    filing = None
    for f in filings:
        report_date = datetime.fromisoformat(f.report_date)
        report_year = (
            report_date.year - 1 if report_date.month <= 3 else report_date.year
        )
        if report_year == year:
            filing = f
            break
    if not filing:
        return None

    try:
        income = (
            XBRLS.from_filings(filings).statements.income_statement().to_dataframe()
        )
    except:
        income = pd.DataFrame()

    url = f"https://www.sec.gov/Archives/edgar/data/{filing.cik}/{filing.accession_no.replace('-', '')}/{filing.primary_document}"

    facts = {
        "Revenues": {
            # 高優先：標準與 ASC 606 定義
            "High": [
                "Revenues",
                "RevenueFromContractWithCustomerExcludingAssessedTax",
                "RevenueFromContractWithCustomerIncludingAssessedTax",
            ],
            # 中優先：銷售細分類（商品、服務）
            "Medium": [
                "SalesRevenueNet",
                "SalesRevenueGoodsNet",
                "SalesRevenueServicesNet",
            ],
            # 特定產業
            "Low": [
                "RevenuesNetOfInterestExpense",
                "RealEstateRevenueNet",
                "RegulatedAndUnregulatedOperatingRevenue",
            ],
            # 最後備援：模擬資料
            # "BusinessAcquisitionsProFormaRevenue",
        },
        "Income": {
            # 高優先：標準最終淨利
            "High": ["NetIncomeLoss", "ProfitLoss"]
            # 中優先：普通股基本淨利
            # "NetIncomeLossAvailableToCommonStockholdersBasic",
        },
    }

    def safe_extract(target):
        def parse_val(val):
            return float(re.sub("[—−]", "-", re.sub(r"[\s,]+", "", str(val))))

        if target == "Revenues":
            levels = ["High", "Medium", "Low"]
        else:
            levels = [None]  # 單層清單

        for level in levels:
            total = 0
            found = []
            fact_list = facts[target][level] if level else facts[target]

            # Income statement
            for fact in fact_list:
                if not income.empty and fact in income.columns:
                    row = income[income["concept"] == fact]
                    for col in income.columns[2:]:
                        col_date = datetime.fromisoformat(col)
                        report_year = (
                            col_date.year if col_date.month > 3 else col_date.year - 1
                        )
                        if report_year == year:
                            val = row[col].values[0]
                            if pd.notna(val):
                                total += float(val)
                                found.append(fact)
                            break
            if found:
                return (total if len(found) > 1 else float(total)), "+".join(found)

            # df["frame"]
            for fact in fact_list:
                col = df[(df["fact"] == fact) & (df["frame"] == f"CY{year}")]
                if not col.empty:
                    val = col.sort_values("filed", ascending=False)["val"].iloc[0]
                    if val:
                        total += parse_val(val)
                        found.append(fact)
            if found:
                return (total if len(found) > 1 else float(total)), "+".join(found)

            # df["end"]
            for fact in fact_list:
                col = df[(df["fact"] == fact) & (df["end"].str.startswith(str(year)))]
                if not col.empty:
                    val = col.sort_values("filed", ascending=False)["val"].iloc[0]
                    if val:
                        total += parse_val(val)
                        found.append(fact)
            if found:
                return (total if len(found) > 1 else float(total)), "+".join(found)

            # Inline XBRL
            try:
                response = requests.get(
                    url, headers={"User-Agent": "ansa1019@gmail.com"}
                )
                soup = BeautifulSoup(response.content, "lxml")
                for fact in fact_list:
                    for tag in soup.find_all(
                        "ix:nonfraction", {"name": f"us-gaap:{fact}"}
                    ):
                        contextref = tag.get("contextref")
                        context = soup.find(id=contextref)
                        end_tag = context.find("xbrli:enddate")
                        report_date = datetime.fromisoformat(end_tag.text.strip())
                        report_year = (
                            report_date.year - 1
                            if report_date.month <= 3
                            else report_date.year
                        )
                        if report_year == year and context.find("xbrli:segment"):
                            val = tag.text
                            if val:
                                total += parse_val(val)
                                found.append(fact)
                if found:
                    return (total if len(found) > 1 else float(total)), "+".join(found)
            except Exception as e:
                print(f"❌ Inline XBRL 抓取失敗：{e}")

        return None

    fact_list = []
    revenues = income_val = None

    result = safe_extract("Revenues")
    if result:
        fact_list.append(result)

    result = safe_extract("Income")
    if result:
        fact_list.append(result)

    return fact_list

In [3]:
# 財務資料
log = ""
financial_data = {}
loop = tqdm(companies)
for ticker in loop:
    searchs = [ticker] + find_cik[ticker] if ticker in find_cik else [ticker]
    for search in searchs:
        try:
            for year in range(report_year[0], report_year[1] + 1):
                loop.set_description(f"正在抓取 {ticker} {year} 年的財務資料...")
                financial_data.setdefault(ticker, {}).setdefault(
                    year, {"Revenues": None, "Net Income": None}
                )
                if (
                    financial_data[ticker][year]["Revenues"] is not None
                    and financial_data[ticker][year]["Net Income"] is not None
                ):
                    continue
                result = extract_financials(search, year)
                if result is not None:
                    revenues, income = result
                    if revenues is not None:
                        financial_data[ticker][year]["Revenues"] = revenues
                    if income is not None:
                        financial_data[ticker][year]["Net Income"] = income
        except Exception as e:
            log += f"❌ 無法取得 {ticker} {year} 年的數據: {e}\n"

# 轉為 DataFrame 並輸出
flat_data = []
for ticker, years in financial_data.items():
    for year, values in years.items():
        row = {
            "ticker": ticker,
            "year": year,
            "Revenues": values.get("Revenues"),
            "Net Income": values.get("Net Income"),
        }
        flat_data.append(row)
df = pd.DataFrame(flat_data)
df.to_csv(financials_file, index=False)
print(f"📁 財務資料已保存到: {financials_file}")

if log:
    print("⚠️ 錯誤紀錄：\n" + log)

正在抓取 LRCX 2014 年的財務資料...: 100%|██████████| 70/70 [14:00<00:00, 12.01s/it] 

📁 財務資料已保存到: financials.csv
⚠️ 錯誤紀錄：
❌ 無法取得 AAPL 2014 年的數據: cannot unpack non-iterable NoneType object
❌ 無法取得 MSFT 2014 年的數據: cannot unpack non-iterable NoneType object
❌ 無法取得 AMZN 2014 年的數據: cannot unpack non-iterable NoneType object
❌ 無法取得 MA 2015 年的數據: cannot unpack non-iterable NoneType object
❌ 無法取得 ORCL 2014 年的數據: 'NoneType' object has no attribute 'columns'
❌ 無法取得 PG 2015 年的數據: cannot unpack non-iterable NoneType object
❌ 無法取得 HD 2014 年的數據: cannot unpack non-iterable NoneType object
❌ 無法取得 ABBV 2014 年的數據: 'NoneType' object has no attribute 'columns'
❌ 無法取得 KO 2014 年的數據: cannot unpack non-iterable NoneType object
❌ 無法取得 MRK 2014 年的數據: cannot unpack non-iterable NoneType object
❌ 無法取得 PEP 2014 年的數據: cannot unpack non-iterable NoneType object
❌ 無法取得 TMO 2014 年的數據: cannot unpack non-iterable NoneType object
❌ 無法取得 ABT 2014 年的數據: 'NoneType' object has no attribute 'columns'
❌ 無法取得 AMD 2014 年的數據: cannot unpack non-iterable NoneType object
❌ 無法取得 PM 2014 年的數據: cannot unpack non-iterable

In [14]:
# 檢查缺值
df = pd.read_csv(financials_file)
df[df.isnull().any(axis=1)]

,ticker,year,Revenues,Net Income
0,AAPL,2014,NaN,NaN
11,MSFT,2014,NaN,NaN
12,AMZN,2014,NaN,NaN
74,MA,2015,NaN,NaN
75,ORCL,2014,NaN,NaN
97,PG,2015,NaN,NaN
98,HD,2014,NaN,NaN
129,ABBV,2014,NaN,NaN
130,KO,2014,NaN,NaN
141,MRK,2014,NaN,NaN


In [16]:
df = pd.read_csv("financials.csv")  # 替換成你的檔名
missing_df = df[df.isnull().any(axis=1)]

results = []

for _, row in tqdm(missing_df.iterrows(), total=len(missing_df)):
    ticker = row["ticker"]
    year = int(row["year"])

    results.append([ticker, year, extract_financials(ticker, year)])
for r in results:
    print(r)

100%|██████████| 39/39 [04:37<00:00,  7.12s/it]

['AAPL', 2014, [(182795000000.0, 'SalesRevenueNet')]]
['MSFT', 2014, [(159781000000.0, 'SalesRevenueNet+SalesRevenueGoodsNet')]]
['AMZN', 2014, [(177976000000.0, 'SalesRevenueNet+SalesRevenueGoodsNet+SalesRevenueServicesNet')]]
['MA', 2015, [(9667000000.0, 'SalesRevenueNet')]]
['ORCL', 2014, [(38226000000.0, 'Revenues')]]
['PG', 2015, [(70749000000.0, 'SalesRevenueNet')]]
['HD', 2014, [(86976000000.0, 'SalesRevenueNet+SalesRevenueServicesNet')]]
['ABBV', 2014, [(19960000000.0, 'SalesRevenueNet')]]
['KO', 2014, [(45998000000.0, 'SalesRevenueGoodsNet')]]
['MRK', 2014, [(42237000000.0, 'SalesRevenueGoodsNet')]]
['PEP', 2014, [(66683000000.0, 'SalesRevenueNet')]]
['TMO', 2014, [(16889600000.0, 'Revenues')]]
['ABT', 2014, [(20247000000.0, 'SalesRevenueNet')]]
['AMD', 2014, [(11012000000.0, 'SalesRevenueNet+SalesRevenueGoodsNet')]]
['PM', 2014, [(80106000000.0, 'SalesRevenueNet')]]
['ISRG', 2014, [(4263400000.0, 'SalesRevenueNet+SalesRevenueGoodsNet+SalesRevenueServicesNet')]]
['INTU', 2014,

In [5]:
year = 2014
cik = "KKR"
company = Company(cik)
filing = company.get_filings(form="10-K", date=f"{year}-04-01:{year+1}-12-31")[-1]
df = company.get_facts().to_pandas()
df = df[df["namespace"] == "us-gaap"]
url = f"https://www.sec.gov/Archives/edgar/data/{filing.cik}/{filing.accession_no.replace('-', '')}/{filing.primary_document}"

result = extract_financials(cik, year)
result

(None, 5872631000.0)

In [6]:
[f for f in df["fact"].unique() if "Sale" in f]

['AvailableForSaleDebtSecuritiesAccumulatedGrossUnrealizedGainBeforeTax',
 'AvailableForSaleDebtSecuritiesAccumulatedGrossUnrealizedLossBeforeTax',
 'AvailableForSaleDebtSecuritiesAmortizedCostBasis',
 'AvailableForSaleSecuritiesDebtMaturitiesAfterFiveThroughTenYearsAmortizedCost',
 'AvailableForSaleSecuritiesDebtMaturitiesAfterFiveThroughTenYearsFairValue',
 'AvailableForSaleSecuritiesDebtMaturitiesAfterOneThroughFiveYearsAmortizedCost',
 'AvailableForSaleSecuritiesDebtMaturitiesAfterOneThroughFiveYearsFairValue',
 'AvailableForSaleSecuritiesDebtMaturitiesAfterTenYearsAmortizedCost',
 'AvailableForSaleSecuritiesDebtMaturitiesAfterTenYearsFairValue',
 'AvailableForSaleSecuritiesDebtMaturitiesSingleMaturityDate',
 'AvailableForSaleSecuritiesDebtMaturitiesSingleMaturityDateAmortizedCostBasis',
 'AvailableForSaleSecuritiesDebtMaturitiesWithinOneYearAmortizedCost',
 'AvailableForSaleSecuritiesDebtMaturitiesWithinOneYearFairValue',
 'AvailableForSaleSecuritiesDebtSecurities',
 'DebtSecuriti